In [1]:
def padding(img, pad_size):
		left_x = img[:,:pad_size,:] # h, w, c
		right_x = img[:,-pad_size:,:]
		y = img.shape[0]
		x = img.shape[1]+(pad_size*2)
		new_x = np.full((y, x, 3),255) # h w c
		new_x[:,:pad_size,:] = right_x
		new_x[:,pad_size:-pad_size,:] = img
		new_x[:,-pad_size:,:] = left_x
		return new_x

def luminance(img):
    r,g,b = img[:,:,2], img[:,:,1], img[:,:,0]
    lum =(0.114*b)+(0.587*g)+(0.299*r)
    mean_lum = np.mean(lum)
    return mean_lum
    
"""
ValueError: could not broadcast input array from shape (224,224,3) into shape (0,224,3)
blank padding (144, 452, 3)
"""
"""
72, 226, 3
(224, 72, 3
"""
def blank_padding(col_dict, img, size:list): #col_dict, q, (224,224)
    print('blank padding',img.shape)
    print(type(size))
    img = cv2.resize(img, (col_dict['size'][0], col_dict['size'][1])) #(36,113,3) into shape (36,224,3)
    print('img shape after resize', img.shape) #img shape (72, 226, 3)
    
    if img.shape[0] > size[0]:
        print('img shape 0',img.shape[0])
        img =cv2.resize(img, (img.shape[1],size[0]))
        print('img smaller than 224 224: A ', img.shape)
    if img.shape[1] > size[1]:
        print('img shape 1',img.shape[1])
        img =cv2.resize(img, (size[1], img.shape[0]))
        print('img smaller than 224 224: B', img.shape)
        #img.shape[1] = size[1]
    w = size[1]

    #w = w.shape[0]
    h = size[0]
    #h = h.shape[0]
    delta_w = w -img.shape[1] #(36, 113, 3)
    delta_h = h-img.shape[0]
    print(w, '-', img.shape[1], '=',delta_w)
    print(h, '-', img.shape[0], '=',delta_h)

    #half_delta_h = int(np.round(delta_h/2, decimals=0))
    #half_delta_w = int(np.round(delta_w/2, decimals=0))
    
    #if delta_h != 0:
    half_delta_h = int(np.floor(delta_h/2))
    #else:
    #    half_delta_h = 1
    #if delta_w != 0:
    half_delta_w = int(np.floor(delta_w/2))
    #else: half_delta_w =0
    #numpy.place(target1_label, target1_label<50, 0)
    
    
    print(delta_h,'   ',half_delta_h )
    print(delta_w,'    ',half_delta_w)
    avg_lum = int(luminance(img))
    new_x = np.full((h,w,3), avg_lum) # 224,224,3
    print('blank padding::',new_x.shape) #could not broadcast input array from shape (36,113,3) into shape (36,114,3)
    if img.shape[1]%2 ==0: # 224
        if img.shape[0]%2 == 0: #72
            if half_delta_w == 0:
                print('g')
                new_x[:,:,:] = img #half_delta_h:-half_delta_h
            else:
                
                """
                (224,224,3) into shape (0,224,3)
                """
                print('a')
                new_x[half_delta_h:-half_delta_h,half_delta_w:-half_delta_w,:] = img
        else:
            print('b')
            new_x[half_delta_h:-(half_delta_h+1),half_delta_w:-half_delta_w,:] = img
    else:
        if img.shape[0]%2 == 0:
            print('c')
            new_x[half_delta_h:-half_delta_h,half_delta_w:-(half_delta_w+1),:] = img #*#*#
        else:
            print('d')
            new_x[half_delta_h:-(half_delta_h+1),half_delta_w:-(half_delta_w+1),:] = img
    #print('blank padding',new_x.shape)


    return new_x

def to_tensor(img):
    im_chan = img.shape[2]
    imgy, imgx = img.shape[0], img.shape[1]
    tensor = torch.tensor(img, dtype=torch.float32)
    tensor = F.normalize(tensor)
    tensor = tensor.permute(2,0,1)
    tensor = tensor.reshape(im_chan,imgy, imgx)
    return tensor
    
def preprocess_im(col_dict, img_path:str):
    img = cv2.imread(img_path) #
    if img is not None:
        #print('1',img.shape)
        img = blank_padding(col_dict, img, (224,224))
        #print('2',img.shape)
        img = to_tensor(img).to(device)
        #print(img.shape)
        return img
    else:
        return img



In [2]:
def blank_padding(col_dict, img, final_size:list): #col_dict, q, (224,224)
    img = cv2.resize(img, (col_dict['size'][0], col_dict['size'][1])) #(36,113,3) into shape (36,224,3)

    if img.shape[0] > final_size[0]:
        img =cv2.resize(img, (img.shape[1],final_size[0]))
        
    if img.shape[1] > final_size[1]:
        img =cv2.resize(img, (final_size[1], img.shape[0]))

    w = final_size[1]
    h = final_size[0]
    
    delta_w = w -img.shape[1]
    delta_h = h-img.shape[0]

    half_delta_h = int(np.floor(delta_h/2))
    half_delta_w = int(np.floor(delta_w/2))

    avg_lum = int(luminance(img))
    new_x = np.full((h,w,3), avg_lum) 
    if img.shape[1]%2 ==0: 
        if img.shape[0]%2 == 0: 
            if half_delta_w == 0:
                new_x[:,:,:] = img 
            else:
                new_x[half_delta_h:-half_delta_h,half_delta_w:-half_delta_w,:] = img
        else:
            new_x[half_delta_h:-(half_delta_h+1),half_delta_w:-half_delta_w,:] = img
    else:
        if img.shape[0]%2 == 0:
            new_x[half_delta_h:-half_delta_h,half_delta_w:-(half_delta_w+1),:] = img #*#*#
        else:
            new_x[half_delta_h:-(half_delta_h+1),half_delta_w:-(half_delta_w+1),:] = img
    return new_x


In [3]:
print(72/2)
print(226/2)

36.0
113.0


In [4]:
import numpy as np
w = 224
print(w/2)
print(w-113)
print((w-113)/2)
print(w%2==0)

w= 224
h = 224
avg_lum = int(4)
delta_w = w -113 #(36, 113, 3)
delta_h = h-36

half_delta_h = int(np.floor(delta_h/2))
half_delta_w = int(np.floor(delta_w/2))

new_x = np.full((h,w,3), avg_lum) # 224,224,3
print('blank padding::',new_x.shape) #could not broadcast input array from shape (36,113,3) into shape (36,114,3)
if 113%2 ==0:
    if 36%2 == 0:
        print('a')
        new_x[half_delta_h:-half_delta_h,half_delta_w:-half_delta_w,:] = np.full((36, 113,3), 0)
    else:
        print('b')
        new_x[half_delta_h:-(half_delta_h+1),half_delta_w:-half_delta_w,:] =np.full((36, 113,3), 0)
else:
    if 36%2 == 0:
        print('c')
        new_x[half_delta_h:-half_delta_h,half_delta_w:-(half_delta_w+1),:] = np.full((36, 113,3), 0)#*#*#
    else:
        print('d')
        new_x[half_delta_h:-(half_delta_h+1),half_delta_w:-(half_delta_w+1),:] =np.full((36, 113,3), 0)

112.0
111
55.5
True
blank padding:: (224, 224, 3)
c


In [5]:
import random
import platform
print(platform.python_version())
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))


3.10.12
numpy==1.26.2
platform==1.0.8


In [6]:
import collections
import json
try:
    with open('testjson.json', 'r') as f:
        #obj = f.read()
        dj = json.load(f, object_pairs_hook= collections.OrderedDict) #obj, 
        print(dj)
except Exception as e:
    print("Error decoding Json")
    print(e)
    
#pp.pprint(dj)

Error decoding Json
[Errno 2] No such file or directory: 'testjson.json'


In [7]:
file_path = r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/'

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


col_dict =dict(
    pad = 5,
    colour = 'colour',
    size = [452,144] #452,144
)


x = None
for file in os.listdir(file_path):
    print(file)
    x = file
    break
print(x)
q = cv2.imread(file_path+'/'+x)

qq = padding(q, 5)

t =blank_padding(col_dict, q, (224,224))

print(t.shape)
plt.imshow(q)
plt.axis = False
plt.show()
plt.imshow(qq)
plt.show()

plt.imshow(t)
plt.axis = False
plt.show()

IDSW004_280423_1357_S_074.JPG_Augmented_left_1.JPG
IDSW004_280423_1357_S_074.JPG_Augmented_left_1.JPG


ValueError: could not broadcast input array from shape (144,224,3) into shape (224,224,3)

In [ ]:
img =q
pad_size = 80
left_x = img[:,:pad_size,:] # h, w, c
plt.imshow(left_x)
plt.show()
right_x = img[:,-pad_size:,:]
plt.imshow(right_x)
plt.show()
y = img.shape[0]
x = img.shape[1]+(pad_size*2)
new_x = np.full((y, x, 3),255) # h w c
plt.imshow(new_x)
plt.show()
new_x[:,:pad_size,:] = right_x
plt.imshow(new_x)
plt.show()
new_x[:,pad_size:-pad_size,:] = img
plt.imshow(new_x)
plt.show()
new_x[:,-pad_size:,:] = left_x